In [1]:
!git clone https://github.com/qwopqwop200/GPTQ-for-LLaMa.git

Cloning into 'GPTQ-for-LLaMa'...
remote: Enumerating objects: 575, done.
remote: Counting objects: 100% (263/263), done.
remote: Compressing objects: 100% (180/180), done.
remote: Total 575 (delta 164), reused 147 (delta 83), pack-reused 312
Receiving objects: 100% (575/575), 413.50 KiB | 2.31 MiB/s, done.
Resolving deltas: 100% (328/328), done.


In [2]:
%%bash

PRESIGNED_URL="https://agi.gpt4.org/llama/LLaMA/*"

MODEL_SIZE="7B"  # edit this list with the model sizes you wish to download
TARGET_FOLDER="./"             # where all files should end up

declare -A N_SHARD_DICT

N_SHARD_DICT["7B"]="0"
N_SHARD_DICT["13B"]="1"
N_SHARD_DICT["30B"]="3"
N_SHARD_DICT["65B"]="7"

echo "Downloading tokenizer"
wget ${PRESIGNED_URL/'*'/"tokenizer.model"} -O ${TARGET_FOLDER}"/tokenizer.model"
wget ${PRESIGNED_URL/'*'/"tokenizer_checklist.chk"} -O ${TARGET_FOLDER}"/tokenizer_checklist.chk"

(cd ${TARGET_FOLDER} && md5sum -c tokenizer_checklist.chk)

for i in ${MODEL_SIZE//,/ }
do
    echo "Downloading ${i}"
    mkdir -p ${TARGET_FOLDER}"/${i}"
    for s in $(seq -f "0%g" 0 ${N_SHARD_DICT[$i]})
    do
        wget ${PRESIGNED_URL/'*'/"${i}/consolidated.${s}.pth"} -O ${TARGET_FOLDER}"/${i}/consolidated.${s}.pth"
    done
    wget ${PRESIGNED_URL/'*'/"${i}/params.json"} -O ${TARGET_FOLDER}"/${i}/params.json"
    wget ${PRESIGNED_URL/'*'/"${i}/checklist.chk"} -O ${TARGET_FOLDER}"/${i}/checklist.chk"
    echo "Checking checksums"
    (cd ${TARGET_FOLDER}"/${i}" && md5sum -c checklist.chk)
done

tokenizer.model: OK
Checking checksums
consolidated.00.pth: OK
params.json: OK


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [3]:
!mv 7B GPTQ-for-LLaMa/7B

In [4]:
%cd GPTQ-for-LLaMa

/content/GPTQ-for-LLaMa


In [8]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117

Looking in indexes: https://download.pytorch.org/whl/cu117, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
import torch
print(torch.__version__)

1.13.1+cu116


In [10]:
!pip install -r requirements.txt

!python3 setup_cuda.py install

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-unka45ro
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-unka45ro
  Resolved https://github.com/huggingface/transformers to commit c07a02a4b7892edfee22cbe57d3cdd9e10ae7a4d
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 40.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 71.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5

In [12]:
!pip install accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.8/212.8 KB 4.3 MB/s eta 0:00:00


In [13]:
!python convert_llama_weights_to_hf.py --input_dir . --model_size 7B --output_dir ./output

Fetching all parameters from the checkpoint at ./7B.
Loading the checkpoint in a Llama model.
Loading checkpoint shards: 100% 33/33 [01:05<00:00,  1.97s/it]
Saving in the Transformers format.
Fetching the tokenizer from ./tokenizer.model.
Traceback (most recent call last):
  File "/content/GPTQ-for-LLaMa/convert_llama_weights_to_hf.py", line 33, in <module>
    main()
  File "/content/GPTQ-for-LLaMa/convert_llama_weights_to_hf.py", line 26, in main
    write_tokenizer(
  File "/usr/local/lib/python3.9/dist-packages/transformers/models/llama/convert_llama_weights_to_hf.py", line 248, in write_tokenizer
    shutil.copyfile(input_tokenizer_path, os.path.join(tokenizer_path, "tokenizer.model"))
  File "/usr/lib/python3.9/shutil.py", line 264, in copyfile
    with open(src, 'rb') as fsrc:
FileNotFoundError: [Errno 2] No such file or directory: './tokenizer.model'


In [17]:
%pwd

'/content/GPTQ-for-LLaMa'

In [36]:
!CUDA_VISIBLE_DEVICES=0 python llama.py ./output/llama-7b ptb --wbits 4 --groupsize 128 --save_safetensors llama7b-4bit-128g.safetensors

hello
second
not loading quant
Loading checkpoint shards: 100% 2/2 [00:44<00:00, 22.27s/it]
third
Found cached dataset ptb_text_only (/root/.cache/huggingface/datasets/ptb_text_only/penn_treebank/1.1.0/8d1b97746fb9765d140e569ec5ddd35e20af4d37761f5e1bf357ea0b081f2c1f)
Found cached dataset ptb_text_only (/root/.cache/huggingface/datasets/ptb_text_only/penn_treebank/1.1.0/8d1b97746fb9765d140e569ec5ddd35e20af4d37761f5e1bf357ea0b081f2c1f)
Traceback (most recent call last):
  File "/content/GPTQ-for-LLaMa/llama.py", line 413, in <module>
    dataloader, testloader = get_loaders(
  File "/content/GPTQ-for-LLaMa/datautils.py", line 109, in get_loaders
    return get_ptb(nsamples, seed, seqlen, model)
  File "/content/GPTQ-for-LLaMa/datautils.py", line 38, in get_ptb
    tokenizer = AutoTokenizer.from_pretrained(model, use_fast=False)
  File "/usr/local/lib/python3.9/dist-packages/transformers/models/auto/tokenization_auto.py", line 680, in from_pretrained
    return tokenizer_class.from_pretra

In [37]:
!CUDA_VISIBLE_DEVICES=0 python llama_inference.py ./output/llama-7b --wbits 4 --groupsize 128 --load llama7b-4bit-128g.pt --text "this is llama"

Loading model ...
Traceback (most recent call last):
  File "/content/GPTQ-for-LLaMa/llama_inference.py", line 112, in <module>
    model = load_quant(args.model, args.load, args.wbits, args.groupsize)
  File "/content/GPTQ-for-LLaMa/llama_inference.py", line 52, in load_quant
    model.load_state_dict(torch.load(checkpoint))
  File "/usr/local/lib/python3.9/dist-packages/torch/serialization.py", line 771, in load
    with _open_file_like(f, 'rb') as opened_file:
  File "/usr/local/lib/python3.9/dist-packages/torch/serialization.py", line 270, in _open_file_like
    return _open_file(name_or_buffer, mode)
  File "/usr/local/lib/python3.9/dist-packages/torch/serialization.py", line 251, in __init__
    super(_open_file, self).__init__(open(name, mode))
FileNotFoundError: [Errno 2] No such file or directory: 'llama7b-4bit-128g.pt'
